Esse código explora o processo de webscrapping na seção de roupas do maior marketplace do Brasil. Por questões de segurança, o nome do site não será divulgado. O dataset resultante do código está disponível na versão crua [aqui](scrapping_roupas_marketplace.csv) e na versão ajustada [aqui](dataset_roupas_marketplacebr_limpo.csv). Também está disponível no kaggle.

## 0. Pré-processamento

In [1]:
#BeautifulSoup e requests para a conexão com o site e scrapping da página, selenium para a simulação do navegador

from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time

In [ ]:
def insere_dados_basicos(elementos, soup):
    '''Encontra os dados básicos das roupas no marketplace. Como
        esses elementos são facilmente selecionáveis, não é necessária
        nenhuma ação além da indicação dos próprios elementos'''
    lista = []
    for item in elementos:
        try:
            elemento = soup.find(item['name'], class_=item['class']).get_text().strip()
            lista.append(elemento)
        except:
            lista.append(None)
        
    return lista

def insere_elementos_tabela(lista_de_textos, soup):
    '''Encontra elementos dentro de uma categorização do tipo
        tabela dentro do site. Procura pelo rótulo de classificação
        (por exemplo, "Marca:"), e seleciona o elemento seguinte
        (por exemplo, o nome da marca)'''
    
    lista = []

    for elemento in lista_de_textos:
        elemento = soup.find('span', text=elemento)
        
        if elemento:
            elemento = elemento.find_next_sibling('span').text.strip()
        else:
            elemento = "N/A"

        lista.append(elemento)
    return lista

def insere_reviews(elemento_nome, elemento_classe, soup):
    '''Circula pelo elemento de reviews da página e seleciona
    cada uma delas, adiciona em uma lista de reviews e devolve a lista'''
    lista_reviews = []
    reviews = soup.find_all(name=elemento_nome, class_=elemento_classe)
    for review in reviews:
        lista_reviews.append(review.get_text().strip())
        
    return lista_reviews

In [2]:
#Ligando o navegador web que vai passar por cada página e obter as informações do scrapping
driver = webdriver.Edge()

In [3]:
# Essa lista indica os valores de index que marcam as passagens de página do site. É algo bem específico do site, onde são 48 itens por página. Assim, o item 49 é a página 2
# O item 49+48 a página 3, e assim por diante. Como são 42 páginas, a lista soma o número 48 por 42 vezes.
start_value = 49
step = 48
n = 42  # Number of times to add the step

range_index = [start_value + i * step for i in range(n)]


## 1. Adquirindo a lista de URLs para o scrapping

In [4]:
#Lista das páginas com os produtos. Por questões de segurança jurídica e privacidade, o nome original do site foi substituído.
lista_paginas = [(f'https://SITE/calcados-roupas-bolsas/roupas_Desde_{index}_NoIndex_True') for index in range_index]

In [5]:
#Headers para fazer as requisições e o soup das páginas
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76",
           "Accept-Language": "pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ja;q=0.5,es;q=0.4"}

In [7]:
#Realizando o request e o soup de cada página a partir da lista de URLs. Em cada página o código passa por cada item do catálogo e copia o link do item para uma lista
links_roupas = []

for url in lista_paginas:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for item in soup.find_all(class_='ui-search-item__group__element shops__items-group-details ui-search-link'):
        links_roupas.append(item.get('href'))
    
    print(f'Número de links: {len(links_roupas)}')

Número de links: 54
Número de links: 108
Número de links: 162
Número de links: 216
Número de links: 270
Número de links: 324
Número de links: 378
Número de links: 432
Número de links: 486
Número de links: 540
Número de links: 594
Número de links: 648
Número de links: 702
Número de links: 756
Número de links: 810
Número de links: 864
Número de links: 918
Número de links: 972
Número de links: 1026
Número de links: 1080
Número de links: 1134
Número de links: 1188
Número de links: 1242
Número de links: 1296
Número de links: 1350
Número de links: 1404
Número de links: 1458
Número de links: 1512
Número de links: 1566
Número de links: 1620
Número de links: 1668
Número de links: 1722
Número de links: 1776
Número de links: 1830
Número de links: 1884
Número de links: 1938
Número de links: 1992
Número de links: 2046
Número de links: 2100
Número de links: 2154
Número de links: 2208
Número de links: 2208


## 2. Realizando o scrapping nas páginas

In [8]:
#As colunas definidas se referem aos nomes das colunas no dataframe que será construído. A lista de texto contém os textos exatos cque são encontrados no site e é utilizada
#Para dar suporte a uma das funções de scrapping. Os elementos básicos listados serão utilizados na função que realiza o scrapping a partir desses mesmos elementos

colunas = ['Título', 'Reais', 'Centavos', 'Condição', 'Nota', 'N_Avaliações', 'Desconto', 'Marca', 'Material', 'Gênero', 'Temporada', 'Review1', 'Review2', 'Review3']
lista_de_textos = ['Marca:', 'Material principal:', 'Gênero:', 'Temporada de lançamento:']
elementos_basicos = [{'name':'h1', 'class':'ui-pdp-title'}, {'name':'span', 'class':'andes-money-amount__fraction'}, {'name':'span', 'class':'andes-money-amount__cents andes-money-amount__cents--superscript-36'},
                     {'name':'span', 'class':'ui-pdp-subtitle'}, {'name':'span', 'class':'ui-pdp-review__rating'}, {'name':'span', 'class':'ui-pdp-review__amount'},
                     {'name':'span', 'class':'andes-money-amount__discount'}]

In [ ]:
#Percorre cada link de roupa e realiza as três funções de scrapping definidas anteriormente para criar 3 listas diferentes. As 3 listas são unidas e adicionadas em um lista de
#listas que será transformada em dataframe.
lista = []

for link in links_roupas:
    page = driver.get(link)
    cod_page = driver.page_source
    soup = BeautifulSoup(cod_page, 'html.parser')
    lista_basicos = insere_dados_basicos(elementos_basicos, soup)
    lista_caracteristica = insere_elementos_tabela(lista_de_textos, soup)
    lista_reviews = insere_reviews('p', 'ui-review-capability-comments__comment__content', soup)
    lista.append(lista_basicos + lista_caracteristica + lista_reviews)
    print(f'n obs: {len(lista)}')
    time.sleep(15) #Tempo para ficar em cada página e evitar o bloqueio de reuisição

In [12]:
#Transforma a lista de listas em um dataframe. Cada lista dentro da lista principal é uma linha, as colunas foram definidas anteriormente.
df = pd.DataFrame(lista, columns=colunas)

In [ ]:
#Salva o dataframe em um arquivo .csv
df.to_csv('scrapping_roupas_marketplace.csv')